# Recurrent Neural Networks

## Part 1: Preprocess the data.
In the csv files, you will find several columns. you only need the "close" column for this exercise


reshape the data! see examples

In [ ]:
# Part 1 - Data Preprocessing

# Importing the libraries


# Importing the training set


# Creating a data structure with 30 timesteps and 1 output


# Reshaping [samples,timesteps,features]
# remember: samples: how many observations you have
#           timesteps: how long does the rnn remember (30 in our case)
#           features: how many variables (1 in our case)


## part 2: model building
Build The RNN, you can use LSTM or GRU

In [ ]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages


# Initialising the RNN


# Compiling the RNN

# Fitting the RNN to the Training set


## Part 3: Making Predictions
make predictions in the Test dataset